In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import pickle
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import hinge_loss
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import precision_recall_curve
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import matplotlib.pyplot as plt
%matplotlib inline

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

/Applications/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
with open('rf03e2.pkl') as f:
    rf03e2 = pickle.load(f)


In [12]:
with open('active_hitters_X3.pkl') as f:
    active_hitters_X3 = pickle.load(f)

with open('active_hitters.pkl') as f:
    active_hitters = pickle.load(f)


In [7]:
active_hitters_X3.head(5)

,R_ratio,H_ratio,HR_ratio,RBI_ratio,SB_ratio,BB_ratio,year,yearID,RBI_filled,SB_filled
136,0.323093,0.339352,0.611266,0.340209,0.456546,0.270966,1,2014,0,0
137,0.187602,0.175629,0.305633,0.175970,0.256807,0.175331,2,2015,0,0
235,0.097411,0.105724,0.114263,0.098681,0.053338,0.089384,1,2013,0,0
236,0.141247,0.183626,0.142829,0.131575,0.133346,0.134076,2,2014,0,0
237,0.211058,0.229996,0.190438,0.175433,0.213353,0.193666,3,2015,0,0


In [14]:
type(active_hitters)

pandas.core.series.Series

In [16]:
predictions = rf03e2.predict(active_hitters_X3)

In [20]:
np.array(active_hitters)[predictions]

array(['alcanar01', 'alcanar01', 'alcanar01', ..., 'alcanar01',
       'alcanar01', 'alcanar01'], dtype=object)